## OpenAPI 활용
1. https://data.go.kr 로그인
2. 기상청 종관 ASOS 일별 관측자료 API

In [2]:
## read_csv
import pandas as pd

In [7]:
df = pd.read_csv('../day01/data/충북푸드뱅크현황.csv', encoding='utf8')
df.tail()

,지역,시군구,센터단위,센터구분,지원센터,신고구분,운영단체명,운영법인분류(대),대표자명,센터전화번호,주소,등록일자
24,충북,충청북도 음성군,기초,푸드뱅크,한울사랑나눔,당연신고,한울사랑나눔푸드뱅크,개인 운영,이*경,043-878-4312,음성군 생극면 음성로 1620번길 7,2011-11-25
25,충북,충청북도 음성군,기초,푸드뱅크,관성푸드뱅크,당연신고,개인,개인 운영,유*호,043-877-9457,음성군 삼성면 대성로547번길 12,2014-04-18
26,충북,충청북도 단양군,기초,푸드뱅크,단양군푸드뱅크,임의신고,단양군사회복지협의회,법인 운영,김*학,043-422-1299,"단양군 단양읍 수변로 83, 3층",2022-12-01
27,충북,충청북도 단양군,기초,푸드뱅크,단양나눔과기쁨,당연신고,기독교대한성결교회(단양수정교회),법인 운영,함*용,043-421-3191,단양군 대강면 장림1길 6,2012-04-17
28,충북,충청북도 단양군,기초,푸드마켓,단양군푸드마켓,임의신고,단양군사회복지협의회,법인 운영,김*학,043-422-1700,"단양군 단양읍 중앙1로 29, 1층",2012-06-22


#### 오픈API
- 기상청 종간 ASOS OpenAPI 신청건 웹브라우저 테스트
- 테스트 성공한 URL
	- https://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=T%2BuGGEG5tU6pCutQzUbmUJgybPjNxikLeWNKpEZyTwBroX0NPIfS6ISH%2Fx%2B8CGwa%2BeRRyW7WpciL%2BrWmQXM49g%3D%3D&pageNo=1&numOfRows=10&dataType=JSON&dataCd=ASOS&dateCd=DAY&startDt=20141001&endDt=20141030&stnIds=108

- urlib, request, parse 모듈 사용 파이썬에서 OPENAPI 호출

In [8]:
# openAPI 호출용 모듈
import requests
import json

In [ ]:
def getAsosDailyData(startdate, enddate):
    url = 'https://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'	#endpoint url
    params = {'serviceKey' : f'T%2BuGGEG5tU6pCutQzUbmUJgybPjNxikLeWNKpEZyTwBroX0NPIfS6ISH%2Fx%2B8CGwa%2BeRRyW7WpciL%2BrWmQXM49g%3D%3D',
              'pageNo':'1',
              'numOfRows':'10',
              'dataType':'JSON',
              'dataCd':'ASOS',
              'dateCd':'DAY',
              'startDt':startdate,
              'endDt' :enddate,
              'stnIds' : '108'}